In [1]:
from census import Census
from us import states
import pandas as pd
import geopandas as gpd
import duckdb as duck

In [2]:
api_key = '19744ba94a6c4a1e1c832a117d3a932a6043451b'

In [20]:
variables = [
'NAME',
'GEO_ID',
'B01001D_001E',
'B01001D_002E',
'B01001D_017E',
'B01001E_001E',
'B01001E_002E',
'B01001E_017E',
'B01001G_001E',
'B01001G_002E',
'B01001G_017E',
'B01001H_001E',
'B01001H_002E',
'B01001H_017E',
'B01001I_001E',
'B01001I_002E',
'B01001I_017E',
'B01002_002E',
'B01002_003E',
'B01003_001E',
'B06011_001E',
'B15003_021E',
'B15003_022E',
'B15003_023E',
'B15003_024E',
'B15003_025E',
'B01001_002E',
'B01001_026E',
'B01001B_001E',
'B01001B_002E',
'B01001B_017E',
'B01001C_001E',
'B01001C_002E',
'B01001C_017E'

]

columns = [
'COUNTY',
'FIPS',
'POPULATION_ASIAN_ALL',
'POPULATION_ASIAN_MALE',
'POPULATION_ASIAN_FEMALE',
'POPULATION_HAWAIIAN_AND_PACIFIC_ISLANDER_ALL',
'POPULATION_HAWAIIAN_AND_PACIFIC_ISLANDER_MALE',
'POPULATION_HAWAIIAN_AND_PACIFIC_ISLANDER_FEMALE',
'POPULATION_TWO_OR_MORE_RACE_ALL',
'POPULATION_TWO_OR_MORE_RACE_MALE',
'POPULATION_TWO_OR_MORE_RACE_FEMALE',
'POPULATION_WHITE_NOT_LATINO_ALL',
'POPULATION_WHITE_NOT_LATINO_MALE',
'POPULATION_WHITE_NOT_LATINO_FEMALE',
'POPULATION_HISPANIC_OR_LATINO_ALL',
'POPULATION_HISPANIC_OR_LATINO_MALE',
'POPULATION_HISPANIC_OR_LATINO_FEMALE',
'AGE_MEDIAN_ALL_MALE',
'AGE_MEDIAN_ALL_FEMALE',
'POPULATION_TOTAL',
'INCOME_MEDIAN_INCOME_12_MONTH',
'EDUCATION_ASSOCIATE_DEGREE',
'EDUCATION_BACHELOR_DEGREE',
'EDUCATION_MASTER_DEGREE',
'EDUCATION_SCHOOL_DEGREE',
'EDUCATION_DOCTORATE_DEGREE',
'GEO_STATE_FIPS',
'GEO_COUNTY_FIPS',
'POPULATION_TOTAL_MALE',
'POPULATION_TOTAL_FEMALE',
'POPULATION_BLACK_ALL',
'POPULATION_BLACK_MALE',
'POPULATION_BLACK_FEMALE',
'POPULATION_NA_OR_AN_ALL',
'POPULATION_NA_OR_AN_MALE',
'POPULATION_NA_OR_AN_FEMALE'

]
df = pd.DataFrame(columns=columns)

In [21]:
year = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

for y in year:
    data = c.acs5.get(variables, {'for': 'county:*'}, year=y)
    df_temp = pd.DataFrame(data)
    df_temp.columns = columns
    df_temp['YEAR'] = y
    df = pd.concat([df, df_temp])

C:\Users\appar\AppData\Local\Temp\ipykernel_99528\3436611790.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_temp])


In [22]:
census_county = df.copy()

In [23]:
census_county.shape

(35426, 37)

In [25]:
# Load the shapefile
counties_raw = gpd.read_file("C:/Users/appar/OneDrive/jup_nb/cb_2018_us_county_500k/cb_2018_us_county_500k.shp",)
counties_raw = counties_raw.to_crs(epsg=2163)
counties = counties_raw.copy()
# Calculate area of each county and sort
counties['area_sqm'] = counties.ALAND + counties.AWATER
counties['area_sqmi'] = counties['area_sqm'] / 2.59e+6  # convert to square miles
counties['area_sqkm'] = counties['area_sqm'] / 10**6

In [26]:
county_size = pd.DataFrame(counties)

In [27]:
county_size.loc[county_size.NAME == 'King']

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,area_sqm,area_sqmi,area_sqkm
1036,53,033,01531933,0500000US53033,53033,King,06,5478600963,497675093,"MULTIPOLYGON (((-1653112.417 530493.679, -1652...",5976276056,2307.442493,5976.276056
2476,48,269,01383920,0500000US48269,48269,King,06,2359148814,6398044,"POLYGON ((-48135.615 -1239308.947, -6617.449 -...",2365546858,913.338555,2365.546858


In [28]:
county_size = county_size.loc[:,['STATEFP', 'COUNTYFP', 'AFFGEOID', 'NAME','ALAND','AWATER','area_sqm', 'area_sqmi','area_sqkm']]

In [29]:
county_size['FIPS'] = county_size['AFFGEOID']

In [30]:
county_size.drop('AFFGEOID', inplace=True, axis=1)

In [31]:
census_data_size = pd.merge(census_county, county_size.loc[:,['FIPS','ALAND','AWATER','area_sqm','area_sqmi','area_sqkm']], how='left')

In [35]:
census_data_size.to_csv('C:/Users/appar/OneDrive/jup_nb/census_data.csv')

In [33]:
county_size.loc[county_size.STATEFP == '09']

,STATEFP,COUNTYFP,NAME,ALAND,AWATER,area_sqm,area_sqmi,area_sqkm,FIPS
56,09,009,New Haven,1565688367,667081242,2232769609,862.073208,2232.769609,0500000US09009
659,09,001,Fairfield,1618651428,549293518,2167944946,837.044381,2167.944946,0500000US09001
660,09,007,Middlesex,956493754,180679140,1137172894,439.062893,1137.172894,0500000US09007
661,09,011,New London,1722716728,276657755,1999374483,771.959260,1999.374483,0500000US09011
1238,09,005,Litchfield,2384116952,62334525,2446451477,944.575860,2446.451477,0500000US09005
1239,09,013,Tolland,1062807467,17549693,1080357160,417.126317,1080.357160,0500000US09013
1991,09,015,Windham,1328478475,21477921,1349956396,521.218686,1349.956396,0500000US09015
2003,09,003,Hartford,1903543897,40543777,1944087674,750.613002,1944.087674,0500000US09003
